In [2]:
import os
import gmsh
gmsh.initialize()               # Need to initialize gmsh
import pygmsh
import meshio
import numpy as np

import matplotlib.pyplot as plt
from scipy import interpolate

## Guitar Mesh Setup

In [9]:
# Guitar Parameters
length = 0.48;
lbWidth = 0.365;
ubWidth = 0.28;
indentWidth = 0.23;
lbLoc = 0.13;
indentLoc = 0.31;
ubLoc = 0.41;
topThickness = 0.003;
backThickness = 0.003;
depth = 0.1;
sideThickness = 0.003;
soundholeDiam = 0.08;
soundholeHeight = 0.3;
neckBlockWidth = 0.08;
endBlockWidth = 0.1;

In [10]:
# Outline of points around the guitar top
x = np.array([0, -endBlockWidth, -lbWidth/2, -indentWidth/2, -ubWidth/2, -neckBlockWidth, 0, neckBlockWidth, ubWidth/2, indentWidth/2, lbWidth/2, endBlockWidth])
y = np.array([0, 0, lbLoc, indentLoc, ubLoc, length, length, length, ubLoc, indentLoc, lbLoc, 0])

# Outline of inner points for the sides
xSide = np.array([0, -endBlockWidth, -lbWidth/2+sideThickness, -indentWidth/2+sideThickness, -ubWidth/2+sideThickness, -neckBlockWidth, 0, neckBlockWidth, ubWidth/2-sideThickness, indentWidth/2-sideThickness, lbWidth/2-sideThickness, endBlockWidth])
ySide = np.array([sideThickness, sideThickness, lbLoc, indentLoc, ubLoc, length-sideThickness, length-sideThickness, length-sideThickness, ubLoc, indentLoc, lbLoc, sideThickness])

# append the starting x,y coordinates
x = np.r_[x, x[0]]
y = np.r_[y, y[0]]

xSide = np.r_[xSide, xSide[0]]
ySide = np.r_[ySide, ySide[0]]

In [159]:
# fit splines to x=f(u) and y=g(u), treating both as periodic. also note that s=0
# is needed in order to force the spline fit to pass through all the input points.
tck, u = interpolate.splprep([x, y],task = 0, k = 5, s=0.0008, per=1)
tckSide, uSide = interpolate.splprep([xSide, ySide],task = 0, k = 5, s=0.0008, per=1)

# evaluate the spline fits for 40 evenly spaced distance values
N = 100;
xi, yi = interpolate.splev(np.linspace(0, 1, N), tck)
xiSide, yiSide = interpolate.splev(np.linspace(0, 1, N), tckSide)

In [160]:
# Makes lists of the points for geometry creation
pointList = []
pointListSideInner = []
pointListSideOuter = []

for i in range(1,N):
    pointList.append([xi[i],yi[i],0.0]) 
    pointListSideInner.append([xiSide[i],yiSide[i],0])
    pointListSideOuter.append([xi[i],yi[i],0])

## Guitar Mesh Construction using gmsh
gmsh API here: https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_9_5/api/gmsh.py

In [161]:
MeshGeometryFolder = os.path.join("MeshGeometry")

In [162]:
gmsh.model.add("Guitar Top")
gmsh.model.setCurrent("Guitar Top")

numPoints = N-1

# top plate boundary points
pointTags = []
for i in range(numPoints):
    gmsh.model.occ.addPoint(pointList[i][0],pointList[i][1],0,tag=i)
    pointTags.append(i)
    
# Make line segments for top plate boundary    
lineTags = []
for j in range(numPoints-1):
    gmsh.model.occ.addLine(j,j+1,tag=j)
    lineTags.append(j)
gmsh.model.occ.addLine(j+1,0,tag=j+1)
lineTags.append(j+1)
    
# Make a curved loop out of the line segments    
gmsh.model.occ.addCurveLoop(lineTags,tag=0)    

# Make a circle for the sound hole
gmsh.model.occ.addCircle(0,soundholeHeight,0,soundholeDiam/2,tag=N)
gmsh.model.occ.addCurveLoop([N],tag=1)    
    
# Make a plane surface from the curved loop    
surf = gmsh.model.occ.addPlaneSurface([0,1],tag=0)    

# Extrude the surface
top = gmsh.model.occ.extrude([(2,surf)],0,0,0.2+topThickness)


gmsh.model.occ.synchronize()


# Add physical group: volume
volumes = gmsh.model.getEntities(dim=3)
assert[volumes == top[0]]
top_volume_marker = 11
gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], top_volume_marker)
gmsh.model.setPhysicalName(volumes[0][0], top_volume_marker, "Top volume")


# Work in progress, label the surfaces, see if the .msh file wil import into dolfinx when surfaces are given phyiscal label
# Add physical group: surface
surfaces = gmsh.model.getEntities(dim=2)
assert[surfaces == top[1]]
top_front_surface, top_back_surface, top_outside_surface, top_soundhole_surface = 1,3,5,7
# gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], top_marker)
# gmsh.model.setPhysicalName(volumes[0][0], top_marker, "Top volume")

# for surface in surfaces:
#     gmsh.model.addPhysicalGroup(2, [surface[1]], top_front_surface)
#     gmsh.model.setPhysicalName(2, top_front_surface, "surface")



# Set mesh
# gmsh.model.mesh.setSize(volumes,0.01)

gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write("testMesh.msh")


# gmsh.model.occ.
# gmsh.model.occ.addPlaneSurface()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 0 (Line)
Info    : [ 10%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
In

## Guitar Mesh Construction (Old)


In [9]:
MeshGeometryFolder = os.path.join("MeshGeometry")

In [10]:
# Create the top mesh
with pygmsh.occ.Geometry() as geom:

    geom.characteristic_length_max = 0.02

    top2D = geom.add_polygon(pointList)
    
    soundhole = geom.add_disk([0.0, soundholeHeight], soundholeDiam/2)
    
    topHole2D = geom.boolean_difference(top2D, soundhole)
    
    top3D = geom.extrude(top2D, [0.0, 0.0, topThickness], num_layers=2)
    
    top3D = [top3D[0], top3D[1]] +  top3D[2]
        
    top_mesh = geom.generate_mesh()

In [11]:
# Create the side mesh
with pygmsh.occ.Geometry() as geom:

    
    geom.characteristic_length_max = 0.02

    sideOuter2D = geom.add_polygon(pointListSideOuter)
    sideInner2D = geom.add_polygon(pointListSideInner)
    
    side2D = geom.boolean_difference(sideOuter2D, sideInner2D)
    
    side3D = geom.extrude(side2D, [0.0, 0.0, depth], num_layers=10)
    
    side_mesh = geom.generate_mesh()

In [12]:
# Create the back mesh
with pygmsh.occ.Geometry() as geom:

    geom.characteristic_length_max = 0.02

    back2D = geom.add_polygon(pointList)
    geom.translate(back2D, [0.0,0.0,depth])
    
    back3D = geom.extrude(back2D, [0.0, 0.0, topThickness], num_layers=2)
    
    back_mesh = geom.generate_mesh()

## Save Mesh to XDMF

In [13]:
# NEED TO FIX

top_mesh_from_msh = meshio.read(os.path.join(MeshGeometryFolder, "top_mesh.msh"))
tetra_top_mesh = create_mesh(top_mesh_from_msh, "tetra")
meshio.write("top_mesh.xdmf", tetra_top_mesh)

# Write the mesh
meshio.write(os.path.join(MeshGeometryFolder,  "top_mesh.xml"), top_mesh)
meshio.write(os.path.join(MeshGeometryFolder,  "side_mesh.xml"), side_mesh)
meshio.write(os.path.join(MeshGeometryFolder,  "back_mesh.xml"), back_mesh)

# Gives Warnings!
# Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.
# Warning: DOLFIN XML can only handle one cell type at a time. Using tetra, discarding vertex, triangle, line.

top_mesh.write(os.path.join(MeshGeometryFolder,  "top_mesh.vtk"))
side_mesh.write(os.path.join(MeshGeometryFolder, "side_mesh.vtk"))
back_mesh.write(os.path.join(MeshGeometryFolder, "back_mesh.vtk"))

NameError: name 'create_mesh' is not defined

In [ ]:
# Documentation
# UFL: https://fenics.readthedocs.io/projects/ufl/en/latest/index.html
# DOLFINx: https://docs.fenicsproject.org/dolfinx/main/python/index.html